[Home](../index.ipynb) / ADC-Wandler: Lösungen
***
<span style="font-size:20pt;">ADC-Wandler: Lösungen</span>

---
# Aufgaben

---
## Aufgabe 1
<div style="background-color:#FFEEEE">
Statt der internen LED soll eine blaue LED an Pin 14 angeschlossen werden (**Vorwiderstand nicht vergessen!**).<br>
   Wie oben soll die LED nur leuchten, solange der Taster an Pin 25 gedrückt ist.<br>
   Zusätzlich soll auf dem Display der An/Aus-Zustand der LED geeignet angezeigt werden.</div>

In [48]:
#####################################
# Aufgabe 1
#####################################

# %serialconnect --port=COM3 # Windows (or %serialconnect --port=COM3 --baud=115200)
%serialconnect # Linux or Windows with one COM port


#====================================
# Constants

ID_BUTTON_ONOFF = 18
ID_LED          = 25

#====================================
# Variables and init

from machine import Pin
import display
import math

display = display.Display()


btnOnOff = Pin( ID_BUTTON_ONOFF, Pin.IN, Pin.PULL_UP )
led      = Pin( ID_LED, Pin.OUT )

display.setCenter( 63, 31 )

def paintOn():
    display.fillCircle( 0, 0, 20 )
    
    for iIndex in range( 12 ):
        display.lineP( 0, 0, iIndex*math.pi/6, 20, 25 )
        
def paintOff():
    display.circle( 0, 0, 20 )
    display.line( -13, -13, 13,  13 )
    display.line( -13,  13, 13, -13 )

bIsOnLast = -1
bIsOn = -1

def paint( bIsOn ):
    global bIsOnLast
    
    if bIsOn != bIsOnLast:
        bIsOnLast = bIsOn
        
        display.clear()
        
        if bIsOn == 0 :
            paintOn()
        else :
            paintOff()
        
        display.show()


led.off()
paint( 1 )


#====================================
# Loop

while True:
    bIsOn = btnOnOff.value()
    
    led.on() if bIsOn == 0 else led.off()

    paint( bIsOn )


Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.
.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 65, in <module>
  File "<stdin>", line 40, in paint
KeyboardInterrupt: 


---
## Aufgabe 2
<div style="background-color:#FFEEEE">Die LED wird mit einem Tastendruck eingeschaltet und beim nächsten Tastendruck wieder ausgeschaltet.</div>

### Anmerkung
Wird der Taster gedrückt **prellt** er ein paar Millisekunden hin und her: es passiert also dauernd, dass nach "Taster gedrückt" sofort ein "Taster nicht gedrückt" und wieder "Taster gedrückt" folgt.

[<img src="images/TasterPrellen.png" width="400">](mages/TasterPrellen.png)

Der Taster muss also **entprellt** werden!  
Am einfachsten geht dies, wenn wir nach dem Drücken so $30 ms$ bis $200 ms$ lang warten und dann erst den Taster wieder abfragen (der Taster des Rotary-Encoders ist recht schlecht, meiner braucht $200 ms$ Entprellzeit!)

z.B. wie in

### Lösung 1

In [ ]:
#####################################
# Aufgabe 2: Loesung 1
#####################################

# %serialconnect --port=COM3 # Windows (or %serialconnect --port=COM3 --baud=115200)
%serialconnect # Linux or Windows with one COM port

#====================================
# Constants

ID_BUTTON_ONOFF = 18
ID_LED          = 25

L_DEBOUNCE_MILLIS = 200 # Entprell-Zeit die gewartet wird, bis wieder eine Abfrage gestartet wird.

STATE_NOT_PRESSED = 0 # Taster ist im Zustand: Nicht gedrückt
STATE_PRESSED     = 1 # Taster ist im Zustand: gedrückt

BUTTON_PRESSED    = 0 # je nach Beschaltung liefert pin.value() ja 0 oder 1.

#====================================
# Variables

lTimeMillisWhenPressd = 0 # Die Zeit beim Tastendruck: nun wird L_DEBOUNCE_MILLIS (hier 30 ms) gewartet bis wieder abgefragt wird.

iPushButtonState = STATE_NOT_PRESSED # Aktueller Zustand des Tasters


from machine import Pin

btnOnOff = Pin( ID_BUTTON_ONOFF, Pin.IN, Pin.PULL_UP )
led      = Pin( ID_LED, Pin.OUT )


#====================================
# Init and loop

from time import ticks_ms, ticks_diff

led.off()
bLEDIsOn = False

while True:
    if iPushButtonState == STATE_PRESSED :
        if ticks_diff( ticks_ms(), lTimeMillisWhenPressd ) > L_DEBOUNCE_MILLIS:
            # After L_DEBOUNCE_MILLIS check if push button is released:
            if btnOnOff.value() != BUTTON_PRESSED :
                iPushButtonState = STATE_NOT_PRESSED
            
    elif btnOnOff.value() == BUTTON_PRESSED :
        iPushButtonState = STATE_PRESSED
        lTimeMillisWhenPressd = ticks_ms()

        # Button was pressed: do action here:
        bLEDIsOn = not bLEDIsOn
        led.on() if bLEDIsOn else led.off()
        

### Lösung 2 mit PushButton-Klasse

Etwas einfacher geht es mit einer schon vorgefertigten `PushButton`-Klasse:  
Diese wird normalerweise schon auf den Controller gespielt, so dass der ganze Code der Klasse wegfällt,  
der verbleibende Code ist dann wirklich sehr kurz!

In [ ]:
#####################################
# Aufgabe 2: Loesung 2
#####################################

# %serialconnect --port=COM3 # Windows (or %serialconnect --port=COM3 --baud=115200)
%serialconnect # Linux or Windows with one COM port

#====================================
# Constants

ID_BUTTON_ONOFF = 18
ID_LED          = 25


#====================================
# Classes

from machine import Pin
import time

class PushButton :
    def __init__( self, _pinNr, _pressedValue = 0, _lTimeMillisWait = 20 ) :
        self._pin = Pin( _pinNr, Pin.IN, Pin.PULL_UP )
        self._pressedValue     = _pressedValue
        self._valueLast        = _pressedValue ^ 1 # self._pin.value() # start with not pressed
        self._lTimeMillisWait  = _lTimeMillisWait
        self._lTimeMillisStart = 0
        self._bWasPressed      = False
        self._isPressedState   = False
        

    def wasPressed( self ):
        self.update()
        
        if self._bWasPressed :
            self._bWasPressed = False
            return True

        else :
            return False
        
        
    def isDown( self ):
        self.update()

        return self._valueLast == self._pressedValue
        
        
    def update( self ):    
        
        if self._isPressedState :
            if time.ticks_diff(time.ticks_ms(), self._lTimeMillisStart ) > self._lTimeMillisWait:
                valueNew = self._pin.value()
                
                if valueNew == self._pressedValue :
                    if valueNew != self._valueLast :
                        self._bWasPressed = True
                else :
                    self._isPressedState = False
                
                self._valueLast = valueNew
                
        else :
            if self._pin.value() != self._valueLast :
                self._isPressedState = True
                self._lTimeMillisStart = time.ticks_ms()

                
#====================================
# Variables and init
            
btnOnOff = PushButton( ID_BUTTON_ONOFF )
led      = Pin( ID_LED, Pin.OUT )

led.off()
bLEDIsOn = False


#====================================
# Loop

while True:
    if btnOnOff.wasPressed():
        bLEDIsOn = not bLEDIsOn
    
        led.on() if bLEDIsOn else led.off()
            

---
## Aufgabe 3

<div style="background-color:#FFEEEE">Wird der Taster <b>lange</b> gedrückt, soll die LED, solange der Taster gedrückt ist, von ihrem aktuellen Helligkeitswert aus langsam zwischen Hell und Dunkel gedimmt werden.<br>
Bei <b>kurzem</b> Betätigen des Tasters soll die LED wie in Aufgabe 2) aus- bzw. auf den letzten Helligkeitswert eingeschaltet werden.</div>
   

In [ ]:
#####################################
# Aufgabe 3
#####################################

# %serialconnect --port=COM3 # Windows (or %serialconnect --port=COM3 --baud=115200)
%serialconnect # Linux or Windows with one COM port


# Warning: Timer0 is used by the functions millis() and sleep() *and* Webserver,
# or if you manually set up timer0, these functions will not work correctly.

#====================================
# Constants

ID_BUTTON_ONOFF = 18
ID_LED          = 25

L_DEBOUNCE_MILLIS  = 200
L_LONGPRESS_MILLIS = 400

PRESSED = 0

STATE_NOT_PRESSED   = 0
STATE_PRESSED       = 1
STATE_PRESSED_SHORT = 2
STATE_PRESSED_LONG  = 3

I_BRIGHTNESS_MIN  =    0
I_BRIGHTNESS_MAX  = 1000
I_BRIGHTNESS_STEP =    4

#====================================
# Variables, functions and init

iBrightnessOn       = I_BRIGHTNESS_MAX 
iBrightnessCur      = I_BRIGHTNESS_MIN

lTimePreviousMillis = 0

iPushButtonState = STATE_NOT_PRESSED
iBrightnessStep  = I_BRIGHTNESS_STEP

#------------------------------------

def toggleLED():
    global iBrightnessCur
    global iBrightnessOn
    iBrightnessCur = iBrightnessOn if iBrightnessCur == I_BRIGHTNESS_MIN else I_BRIGHTNESS_MIN

    led.duty( iBrightnessCur )


def dimmLED():
    global iBrightnessCur
    global iBrightnessOn
    global iBrightnessStep
    
    iBrightnessOn += iBrightnessStep
    iBrightnessCur = iBrightnessOn

    if iBrightnessOn <= I_BRIGHTNESS_MIN :
        iBrightnessOn = I_BRIGHTNESS_MIN
        iBrightnessStep *= -1
        
    elif iBrightnessOn >= I_BRIGHTNESS_MAX :
        iBrightnessOn = I_BRIGHTNESS_MAX
        iBrightnessStep *= -1

    led.duty( iBrightnessCur )
    
    time.sleep_ms( 20 )    
    
#------------------------------------
    
from machine import PWM, Pin
from time import ticks_diff, ticks_ms, sleep

btnOnOff = Pin( ID_BUTTON_ONOFF, Pin.IN, Pin.PULL_UP )
led      = PWM( Pin(ID_LED, Pin.OUT), freq=1000 ) # 25 HelTec, 13 Croduino
sleep( 0.1 ) # Patch: sleep needed: PWM( Pin(ID_LED, Pin.OUT), freq=1000, duty=iBrightnessCur ) does not work, also led.duty( iBrightnessCur ) without sleep! (2022-02-14)
led.duty( iBrightnessCur )


#====================================
# Loop

try:
    while True:
        if iPushButtonState == STATE_NOT_PRESSED :
            if btnOnOff.value() == PRESSED :
                iPushButtonState = STATE_PRESSED
                lTimePreviousMillis = time.ticks_ms()


        elif iPushButtonState == STATE_PRESSED :
            lTimeDiff = ticks_diff(ticks_ms(), lTimePreviousMillis)

            if lTimeDiff > L_LONGPRESS_MILLIS :
                iPushButtonState = STATE_PRESSED_LONG

            elif lTimeDiff > L_DEBOUNCE_MILLIS :
                if btnOnOff.value() != PRESSED :
                    iPushButtonState = STATE_PRESSED_SHORT
                    
        elif iPushButtonState == STATE_PRESSED_SHORT :
             toggleLED() # do action
             # button was released, so set state not pressed
             iPushButtonState = STATE_NOT_PRESSED
            
        elif iPushButtonState == STATE_PRESSED_LONG :
            dimmLED();  # do action

            if  btnOnOff.value() != PRESSED : # check if button is released
                iPushButtonState = STATE_NOT_PRESSED

finally:
    led.deinit() # needed, otherwise You have to reconnect the controller
